In [11]:
import zipfile
import os

# Define the path to the zip file and the target directory
zip_path = 'llm_project (1).zip'
extract_dir = 'data'

# Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to: {extract_dir}")

Files extracted to: data


In [13]:
ls data

ai_generated_train_essays.csv        test_essays.csv      train_prompts.csv
ai_generated_train_essays_gpt-4.csv  train_essays.csv
sample_submission.csv                train_essays_v1.csv


In [10]:


import warnings
warnings.simplefilter('ignore')

In [4]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 3.4 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd

In [8]:
df_ess = pd.read_csv("data/train_essays.csv")
df_ess

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0
1375,ffa247e0,0,There's a new trend that has been developing f...,0
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0


In [9]:
df_pro = pd.read_csv("data/train_prompts.csv")
df_pro

,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...


In [9]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/home/user933/Desktop/asds/LLM-Detect-AI-Generated-Text/data/combined_df_with_source.csv')

# Check for NaN values in the 'generated' column
nan_in_generated = df['generated'].isna().sum()

print(f"Number of NaN values in 'generated' column: {nan_in_generated}")

Number of NaN values in 'generated' column: 0


In [10]:
# check the ddf duplicates count
duplicates_count = df.duplicated().sum()

In [11]:
duplicates_count

np.int64(259396)

In [12]:
df.shape

(845982, 3)

In [13]:
df = df.drop_duplicates(subset=['text'])

In [14]:
df.shape

(493704, 3)

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split


# Create stratify key
df['stratify_key'] = df['source'].astype(str) + "_" + df['generated'].astype(str)

# Split the data
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df['stratify_key'],
    random_state=42
)

# Drop the helper column from both train and test sets
train_df.drop(columns='stratify_key', inplace=True)
test_df.drop(columns='stratify_key', inplace=True)

# Save to CSV files
train_df.to_csv('/home/user933/Desktop/asds/LLM-Detect-AI-Generated-Text/data/new_train.csv', index=False)
test_df.to_csv('/home/user933/Desktop/asds/LLM-Detect-AI-Generated-Text/data/new_test.csv', index=False)
    

In [12]:
import spacy
import pandas as pd

# Load English pipeline
nlp = spacy.load("en_core_web_sm")

PRONOUNS = {"i", "we", "you", "he", "she", "they", "it", "me", "us", "him", "her", "them"}

def extract_style_features_spacy(df, text_col="text"):
    rows = []

    for doc in nlp.pipe(df[text_col], batch_size=32):
        words = [token.text for token in doc if token.is_alpha]
        num_words = len(words)
        num_chars = len(doc.text)
        num_sentences = len(list(doc.sents))
        unique_words = set(words)

        row = {
            "avg_word_len": sum(len(w) for w in words) / num_words if num_words else 0,
            "avg_sentence_len": num_words / num_sentences if num_sentences else 0,
            "num_sentences": num_sentences,
            "num_words": num_words,
            "num_chars": num_chars,
            "lexical_diversity": len(unique_words) / num_words if num_words else 0,
            "num_exclamations": doc.text.count("!"),
            "num_questions": doc.text.count("?"),
            "num_commas": doc.text.count(","),
            "num_semis": doc.text.count(";"),
            "num_pronouns": sum(1 for token in doc if token.text.lower() in PRONOUNS)
        }

        rows.append(row)

    return pd.DataFrame(rows)


In [6]:
df = pd.read_csv("data/train_dataset.csv")
style_feats = extract_style_features_spacy(df)

# Combine with label
final_df = pd.concat([df["text"], style_feats, df["generated"]], axis=1)
final_df.to_csv("train_with_spacy_features.csv", index=False)

In [13]:
df = pd.read_csv("data/texts_labled.csv")
style_feats = extract_style_features_spacy(df)

# Combine with label
final_df = pd.concat([df["text"], style_feats, df["generated"]], axis=1)
final_df.to_csv("manual_test_data.csv", index=False)

In [14]:
final_df

,text,avg_word_len,avg_sentence_len,num_sentences,num_words,num_chars,lexical_diversity,num_exclamations,num_questions,num_commas,num_semis,num_pronouns,generated
0,"When talking about working, some people always...",4.303571,14.000000,24,336,1833,0.529762,0,1,16,1,40,0
1,People often question why we spend such a sign...,4.844011,15.608696,23,359,2154,0.582173,0,1,29,0,19,1
2,Work is not merely an individual endeavor—it i...,5.070336,12.576923,26,327,2035,0.629969,0,0,19,0,14,1
3,Many people believe that work is just a means ...,4.831081,12.869565,23,296,1790,0.638514,0,0,25,1,13,1
4,"At the outside, it might seem obvious that job...",4.635945,22.842105,19,434,2482,0.518433,0,0,13,0,16,0
5,Many people hold the view that face-to-face co...,4.459547,16.263158,19,309,1721,0.511327,0,0,16,0,17,0
6,"With the rapid growth of high technology, comp...",4.859848,20.307692,13,264,1573,0.541667,0,0,13,1,12,0
7,"Success, in its truest form, is often the resu...",5.356589,17.200000,15,258,1676,0.674419,0,0,23,0,4,1
8,"Success rarely comes to those who sit idle, dr...",4.607287,14.529412,17,247,1428,0.668016,0,0,15,0,12,1
9,The debate between taking risks and planning c...,4.879828,13.705882,17,233,1407,0.678112,0,0,13,1,8,1


In [9]:
df = pd.read_csv("data/test_dataset.csv")
style_feats = extract_style_features_spacy(df)

# Combine with label
final_df = pd.concat([df["text"], style_feats, df["generated"]], axis=1)
final_df.to_csv("test_with_spacy_features.csv", index=False)

In [11]:
final_df.columns

Index(['text', 'avg_word_len', 'avg_sentence_len', 'num_sentences',
       'num_words', 'num_chars', 'lexical_diversity', 'num_exclamations',
       'num_questions', 'num_commas', 'num_semis', 'num_pronouns',
       'generated'],
      dtype='object')